In [1]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
model,tokenizer=FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.3-70B-Instruct",    
    max_seq_length=1024,
    load_in_4bit=True
    )

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.096 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [3]:
#%store tokenizer
#%store model
# Stored 'model' (LlamaForCausalLM)
#Stored 'tokenizer' (PreTrainedTokenizerFast)


In [4]:
model=FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None     
)

#%store model

tokenizer=get_chat_template(
    tokenizer,
    chat_template="llama-3.1"
)

%store tokenizer


Unsloth 2026.2.1 patched 80 layers with 80 QKV layers, 80 O layers and 80 MLP layers.


Stored 'tokenizer' (PreTrainedTokenizerFast)


In [7]:
def prompt_format(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }


from datasets import load_dataset
from unsloth.chat_templates import standardize_sharegpt

dataset = load_dataset("pranavpande01/delhi-food", split = "train")
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(prompt_format, batched = True,)

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

In [10]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq

trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_sequence_length=1024,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"

    )
)

Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/53 [00:00<?, ? examples/s]

In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=20):   0%|          | 0/53 [00:00<?, ? examples/s]

Filter (num_proc=20):   0%|          | 0/53 [00:00<?, ? examples/s]

In [14]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                Chandni Chowk, designed by Jahanara Begum, offers a busy commercial experience with the tranquil Digambar Jain Temple and its associated Bird Hospital. Nearby, the Gauri Shankar Temple houses an ancient lingam.\n\nIs the Bird Hospital still actively treating injured birds, and what kind of species are typically brought there?<|eot_id|>'

In [16]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")
trainer_stats = trainer.train()

GPU = NVIDIA H100 80GB HBM3. Max memory = 79.096 GB.
39.217 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 53 | Num Epochs = 9 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 207,093,760 of 70,760,800,256 (0.29% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.170200
2,2.340300
3,2.281200
4,2.259600
5,1.832000
6,1.809300
7,1.810600
8,1.405500
9,1.405300
10,1.623800


In [17]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Here's a conversation-initiating statement based on the knowledge provided, with varied starts and a natural, specific feel:\n\n**\"Hey, have you ever tried that place with the two brothers who make customized masalas? I was thinking of checking them out for their juices or maybe even their fruit chaat – I heard it's only ₹50 or ₹60, and I'm craving something refreshing!\"**"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHere\'s a conversation-initiating statement based on the knowledge provided, with varied starts and a natural, specific feel:\n\n**"Hey, have you ever tried that place with the two brothers who make customized masalas? I was thinking of checking them out for their juices or maybe even their fruit chaat – I heard it\'s only ₹50 or ₹60, and I\'m craving something refreshing!"**<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThese brothers offer customized masalas and are known for their refreshing juices, shakes, lassi, and masala soda, alongside their popular aloo chaat and fruit chaat. They are closed on Sundays, with fruit chaat priced at INR 50 or 60 and aloo chaat at INR']